# Parking Detection Experiments

## Quick Start:
1. Set model path and video path
2. Run cells
3. Results saved automatically to `../output/experiments/`

In [21]:
import cv2
import numpy as np
from ultralytics import YOLO
import json
import os
import time
import pandas as pd
from datetime import datetime
from collections import defaultdict

class ParkingDetector:
    def __init__(self, model_path, video_path):
        self.model_path = model_path
        self.video_path = video_path
        self.model_name = os.path.splitext(os.path.basename(model_path))[0]
        self.video_name = os.path.splitext(os.path.basename(video_path))[0]
        
        # Find parking areas file
        self.areas_file = f'../output/parking-areas-{self.video_name}.json'
        
        # Results
        self.results = {
            'experiment_id': f"{self.model_name}_{self.video_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            'model_path': model_path,
            'video_path': video_path,
            'areas_file': self.areas_file,
            'fps_data': [],
            'parked_vehicles': [],
            'tracking_issues': [],  # New: track flickering/lost detections
            'detection_stats': {},
            'performance': {}
        }
        
        print(f"Model: {self.model_name}")
        print(f"Video: {self.video_name}")
        print(f"Looking for areas: {self.areas_file}")
        
    def load_model_and_areas(self):
        # Load YOLO model
        print("\nLoading model...")
        self.model = YOLO(self.model_path)
        print(f"Model loaded: {self.model_name}")
        
        # Load parking areas
        if not os.path.exists(self.areas_file):
            print(f"Parking areas not found: {self.areas_file}")
            print("   Run define-park-areas.ipynb first!")
            return False
            
        with open(self.areas_file, 'r') as f:
            areas_data = json.load(f)
            
        self.parking_areas = [np.array(area) for area in areas_data['parking_areas']]
        print(f"Loaded {len(self.parking_areas)} parking areas")
        return True
        
    def is_in_parking_area(self, center_point):
        for i, area in enumerate(self.parking_areas):
            if cv2.pointPolygonTest(area, center_point, False) >= 0:
                return i
        return -1
        
    def run_detection(self, show_video=True):
        if not self.load_model_and_areas():
            return False
            
        print("\nStarting detection...")
        
        cap = cv2.VideoCapture(self.video_path)
        if not cap.isOpened():
            print(f"Cannot open video: {self.video_path}")
            return False
            
        # Video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Tracking
        parking_timers = defaultdict(lambda: None)
        parked_vehicles = set()
        vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck
        parking_threshold = 30  # seconds
        
        # Flickering/tracking issue detection
        last_seen = {}  # track_id -> frame_count when last seen
        lost_vehicles = {}  # track_id -> frame_count when lost
        tracking_gaps = defaultdict(list)  # track_id -> list of (lost_frame, found_frame) gaps
        
        # Stats
        frame_count = 0
        total_detections = 0
        start_time = time.time()
        fps_times = []
        
        print(f"Video: {total_frames} frames, {fps:.1f} FPS")
        if show_video:
            print("Press 'q' to stop early, 's' to toggle video display")
        else:
            print("Processing without video display (faster)")
        print()
        
        display_video = show_video
        
        while True:
            frame_start = time.time()
            
            ret, frame = cap.read()
            if not ret:
                break
                
            frame_count += 1
            current_time = frame_count / fps
            
            # Run detection
            results = self.model.track(frame, persist=True, classes=vehicle_classes, verbose=False)
            
            # Draw parking areas on frame
            if display_video:
                display_frame = frame.copy()
                for i, area in enumerate(self.parking_areas):
                    cv2.polylines(display_frame, [area], True, (0, 255, 0), 2)
                    # Add area label
                    center = area.mean(axis=0).astype(int)
                    cv2.putText(display_frame, f"Area {i+1}", tuple(center), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Process detections
            current_frame_vehicles = set()
            if results[0].boxes is not None and results[0].boxes.id is not None:
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
                confidences = results[0].boxes.conf.float().cpu().tolist()
                
                for box, track_id, conf in zip(boxes, track_ids, confidences):
                    if conf < 0.5:
                        continue
                        
                    total_detections += 1
                    current_frame_vehicles.add(track_id)
                    
                    # Track flickering: check if this vehicle was lost and now found
                    if track_id in lost_vehicles:
                        lost_frame = lost_vehicles[track_id]
                        gap_duration = frame_count - lost_frame
                        tracking_gaps[track_id].append({
                            'lost_frame': lost_frame,
                            'found_frame': frame_count,
                            'gap_duration': gap_duration,
                            'gap_time_seconds': gap_duration / fps
                        })
                        print(f"Vehicle {track_id} re-tracked after {gap_duration} frames ({gap_duration/fps:.1f}s)")
                        del lost_vehicles[track_id]
                    
                    # Update last seen
                    last_seen[track_id] = frame_count
                    
                    # Get vehicle center and bounding box
                    x, y, w, h = box
                    center = (int(x), int(y))
                    
                    # Draw detection on frame
                    if display_video:
                        x1, y1 = int(x - w/2), int(y - h/2)
                        x2, y2 = int(x + w/2), int(y + h/2)
                        
                        # Check if in parking area
                        area_id = self.is_in_parking_area(center)
                        
                        if area_id >= 0:  # In parking area
                            # Check if vehicle is already marked as parked
                            if track_id in parked_vehicles:
                                color = (0, 0, 255)  # Red for parked vehicles
                                label = f"ID:{track_id} PARKED Area:{area_id+1}"
                            else:
                                color = (0, 255, 255)  # Yellow for vehicles in parking area but not yet parked
                                label = f"ID:{track_id} Area:{area_id+1}"
                        else:
                            color = (255, 0, 0)  # Blue for other vehicles
                            label = f"ID:{track_id}"
                            
                        cv2.rectangle(display_frame, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(display_frame, label, (x1, y1-10), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    
                    # Check if in parking area
                    area_id = self.is_in_parking_area(center)
                    
                    if area_id >= 0:  # In parking area
                        if track_id not in parking_timers:
                            parking_timers[track_id] = current_time
                            
                        # Check if parked long enough
                        parked_duration = current_time - parking_timers[track_id]
                        
                        if parked_duration >= parking_threshold and track_id not in parked_vehicles:
                            parked_vehicles.add(track_id)
                            
                            # Save parked vehicle data
                            self.results['parked_vehicles'].append({
                                'track_id': track_id,
                                'area_id': area_id + 1,
                                'time': current_time,
                                'frame': frame_count,
                                'duration': parked_duration
                            })
                            print(f"Vehicle {track_id} parked in area {area_id+1} after {parked_duration:.1f}s")
                    else:
                        # Vehicle left parking area
                        if track_id in parking_timers:
                            del parking_timers[track_id]
                        if track_id in parked_vehicles:
                            parked_vehicles.remove(track_id)
            
            # Detect lost vehicles (were seen recently but not in current frame)
            for vehicle_id, last_frame in list(last_seen.items()):
                if vehicle_id not in current_frame_vehicles and (frame_count - last_frame) == 1:
                    # Vehicle just lost tracking
                    lost_vehicles[vehicle_id] = frame_count
                    print(f"Vehicle {vehicle_id} lost tracking at frame {frame_count}")
            
            # Display video frame
            if display_video:
                # Add info overlay
                info_text = [
                    f"Frame: {frame_count}/{total_frames}",
                    f"FPS: {processing_fps:.1f}" if 'processing_fps' in locals() else "FPS: calculating...",
                    f"Detections: {total_detections}",
                    f"Parked: {len(parked_vehicles)}"
                ]
                
                for i, text in enumerate(info_text):
                    cv2.putText(display_frame, text, (10, 30 + i*25), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                cv2.imshow('Parking Detection', display_frame)
            
            # Calculate FPS
            frame_time = time.time() - frame_start
            processing_fps = 1.0 / frame_time if frame_time > 0 else 0
            fps_times.append(processing_fps)
            
            # Show progress
            if frame_count % 30 == 0:
                progress = (frame_count / total_frames) * 100
                avg_fps = sum(fps_times[-30:]) / min(30, len(fps_times))
                print(f"Progress: {progress:.1f}% | FPS: {avg_fps:.1f} | Parked: {len(parked_vehicles)}")
            
            # Check for quit or display toggle
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("Stopped by user")
                break
            elif key == ord('s'):
                display_video = not display_video
                if display_video:
                    print("Video display enabled")
                else:
                    print("Video display disabled")
                    cv2.destroyAllWindows()
                
        cap.release()
        cv2.destroyAllWindows()
        
        # Calculate final stats
        total_time = time.time() - start_time
        avg_fps = sum(fps_times) / len(fps_times) if fps_times else 0
        
        # Process tracking issues data
        total_gaps = sum(len(gaps) for gaps in tracking_gaps.values())
        vehicles_with_gaps = len(tracking_gaps)
        
        # Convert tracking gaps to list format for JSON serialization
        tracking_issues_list = []
        for vehicle_id, gaps in tracking_gaps.items():
            for gap in gaps:
                tracking_issues_list.append({
                    'track_id': vehicle_id,
                    'lost_frame': gap['lost_frame'],
                    'found_frame': gap['found_frame'],
                    'gap_duration_frames': gap['gap_duration'],
                    'gap_duration_seconds': gap['gap_time_seconds']
                })
        
        self.results['fps_data'] = fps_times
        self.results['tracking_issues'] = tracking_issues_list
        self.results['detection_stats'] = {
            'total_frames': frame_count,
            'total_detections': total_detections,
            'unique_parked': len(parked_vehicles),
            'processing_time': total_time,
            'tracking_gaps_total': total_gaps,
            'vehicles_with_tracking_issues': vehicles_with_gaps
        }
        self.results['performance'] = {
            'avg_fps': avg_fps,
            'min_fps': min(fps_times) if fps_times else 0,
            'max_fps': max(fps_times) if fps_times else 0
        }
        
        print(f"\nDetection completed!")
        print(f"Processed {frame_count} frames in {total_time:.1f}s")
        print(f"Average FPS: {avg_fps:.1f}")
        print(f"Parked vehicles detected: {len(parked_vehicles)}")
        print(f"Tracking issues: {total_gaps} gaps in {vehicles_with_gaps} vehicles")
        
        return True
        
    def save_results(self):
        # Create output directory
        output_dir = f"../output/experiments/{self.results['experiment_id']}"
        os.makedirs(output_dir, exist_ok=True)
        
        # Save JSON results
        json_file = f"{output_dir}/results.json"
        with open(json_file, 'w') as f:
            json.dump(self.results, f, indent=2, default=str)
            
        # Save CSV summary
        csv_file = f"{output_dir}/summary.csv"
        summary_data = {
            'experiment_id': [self.results['experiment_id']],
            'model': [self.model_name],
            'video': [self.video_name],
            'avg_fps': [self.results['performance']['avg_fps']],
            'min_fps': [self.results['performance']['min_fps']],
            'max_fps': [self.results['performance']['max_fps']],
            'total_frames': [self.results['detection_stats']['total_frames']],
            'total_detections': [self.results['detection_stats']['total_detections']],
            'parked_vehicles': [self.results['detection_stats']['unique_parked']],
            'processing_time': [self.results['detection_stats']['processing_time']],
            'tracking_gaps': [self.results['detection_stats']['tracking_gaps_total']],
            'vehicles_with_issues': [self.results['detection_stats']['vehicles_with_tracking_issues']]
        }
        
        df = pd.DataFrame(summary_data)
        df.to_csv(csv_file, index=False)
        
        # Save parked vehicles details
        if self.results['parked_vehicles']:
            parked_df = pd.DataFrame(self.results['parked_vehicles'])
            parked_df.to_csv(f"{output_dir}/parked_vehicles.csv", index=False)
            
        # Save tracking issues details
        if self.results['tracking_issues']:
            tracking_df = pd.DataFrame(self.results['tracking_issues'])
            tracking_df.to_csv(f"{output_dir}/tracking_issues.csv", index=False)
            
        print(f"\nResults saved to: {output_dir}")
        files_list = "results.json, summary.csv"
        if self.results['parked_vehicles']:
            files_list += ", parked_vehicles.csv"
        if self.results['tracking_issues']:
            files_list += ", tracking_issues.csv"
        print(f"Files: {files_list}")
        
        return output_dir

In [22]:
# EXPERIMENT CONFIGURATION

# Model and dataset paths
model_path = '../yolo-model/yolov8n.pt'
video_path = '../dataset/video_1_cut50-00_01-00-00.mov'

print(f"Configuration:")
print(f"   Model: {model_path}")
print(f"   Video: {video_path}")

# Check files exist
if not os.path.exists(model_path):
    print(f"Model not found: {model_path}")
elif not os.path.exists(video_path):
    print(f"Video not found: {video_path}")
else:
    print(f"Files found, ready to run!")

Configuration:
   Model: ../yolo-model/yolov8n.pt
   Video: ../dataset/video_1_cut50-00_01-00-00.mov
Files found, ready to run!


In [23]:
# RUN DETECTION EXPERIMENT

detector = ParkingDetector(model_path, video_path)

if detector.run_detection(show_video=True):  # Set to False for faster processing without video
    output_dir = detector.save_results()
    print(f"\nExperiment completed successfully!")
    print(f"Check results in: {output_dir}")
else:
    print(f"\nExperiment failed")
    print(f"   Make sure you've run define-park-areas.ipynb first")

Model: yolov8n
Video: video_1_cut50-00_01-00-00
Looking for areas: ../output/parking-areas-video_1_cut50-00_01-00-00.json

Loading model...
Model loaded: yolov8n
Loaded 2 parking areas

Starting detection...
Video: 18000 frames, 30.0 FPS
Press 'q' to stop early, 's' to toggle video display

Vehicle 5 lost tracking at frame 2
Vehicle 6 lost tracking at frame 2
Vehicle 7 lost tracking at frame 2
Vehicle 11 lost tracking at frame 4
Vehicle 5 lost tracking at frame 2
Vehicle 6 lost tracking at frame 2
Vehicle 7 lost tracking at frame 2
Vehicle 11 lost tracking at frame 4
Vehicle 11 re-tracked after 1 frames (0.0s)
Vehicle 11 re-tracked after 1 frames (0.0s)
Vehicle 14 lost tracking at frame 8
Vehicle 14 lost tracking at frame 8
Vehicle 2 lost tracking at frame 12
Vehicle 2 re-tracked after 2 frames (0.1s)
Vehicle 12 lost tracking at frame 14
Vehicle 2 lost tracking at frame 12
Vehicle 2 re-tracked after 2 frames (0.1s)
Vehicle 12 lost tracking at frame 14
Vehicle 7 re-tracked after 17 fram

In [ ]:
# QUICK RESULTS DISPLAY

if 'detector' in locals() and detector.results['detection_stats']:
    print("\nEXPERIMENT RESULTS")
    print("=" * 40)
    
    stats = detector.results['detection_stats']
    perf = detector.results['performance']
    
    print(f"Model: {detector.model_name}")
    print(f"Video: {detector.video_name}")
    print(f"Processing Time: {stats['processing_time']:.1f}s")
    print(f"Frames Processed: {stats['total_frames']}")
    print(f"Average FPS: {perf['avg_fps']:.1f}")
    print(f"Total Detections: {stats['total_detections']}")
    print(f"Parked Vehicles: {stats['unique_parked']}")
    print(f"Tracking Issues: {stats.get('tracking_gaps_total', 0)} gaps in {stats.get('vehicles_with_tracking_issues', 0)} vehicles")
    
    if detector.results['parked_vehicles']:
        print(f"\nParked Vehicle Details:")
        for pv in detector.results['parked_vehicles']:
            print(f"   Vehicle {pv['track_id']}: Area {pv['area_id']}, {pv['duration']:.1f}s")
    
    if detector.results['tracking_issues']:
        print(f"\nTracking Issues Details:")
        for issue in detector.results['tracking_issues']:
            print(f"   Vehicle {issue['track_id']}: Lost at frame {issue['lost_frame']}, found at {issue['found_frame']} ({issue['gap_duration_seconds']:.1f}s gap)")
else:
    print("Run the detection experiment first!")

    #test


EXPERIMENT RESULTS
Model: yolov8n
Video: video_1_cut50-00_01-00-00
Processing Time: 937.6s
Frames Processed: 11643
Average FPS: 15.3
Total Detections: 65241
Parked Vehicles: 15
Tracking Issues: 2942 gaps in 483 vehicles

Parked Vehicle Details:
   Vehicle 8: Area 2, 30.0s
   Vehicle 3: Area 2, 30.0s
   Vehicle 11: Area 1, 30.0s
   Vehicle 225: Area 2, 30.0s
   Vehicle 385: Area 1, 32.6s
   Vehicle 912: Area 2, 30.2s
   Vehicle 943: Area 2, 30.0s
   Vehicle 1447: Area 1, 30.0s
   Vehicle 2007: Area 1, 30.0s
   Vehicle 2063: Area 1, 33.7s
   Vehicle 2729: Area 2, 30.0s
   Vehicle 4231: Area 1, 30.0s
   Vehicle 4256: Area 1, 44.0s
   Vehicle 6012: Area 1, 30.0s
   Vehicle 6055: Area 1, 30.5s

Tracking Issues Details:
   Vehicle 11: Lost at frame 4, found at 5 (0.0s gap)
   Vehicle 11: Lost at frame 496, found at 504 (0.3s gap)
   Vehicle 11: Lost at frame 506, found at 561 (1.8s gap)
   Vehicle 11: Lost at frame 562, found at 566 (0.1s gap)
   Vehicle 11: Lost at frame 607, found at 619 